# 4. waterfall workflow - 多智能体顺序执行的工作流程

首先将 wde 目录加入 python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入需要的模块

In [2]:
from wde.agents import RolePlayingAgent
from wde.agents import SummaryAgent
from wde.agents import Session
from wde.agents import UserInput

设定 llm 参数

In [3]:
llm_config = {"model": "Qwen/Qwen2-7B-Instruct", "type": "zeroclient"}

下面演示的登机场景，第一名服务员询问客户姓名和目的地，第二名服务员询问客户喜好，第三名服务员根据客户个人信息和喜好跟客户聊天打发时间。

In [4]:
onboarding_personal_information_agent = RolePlayingAgent(
    name="Onboarding Personal Information Agent",
    system_message='''您是一位乐于助人的客户引导代理，
    您在这里是为了帮助新客户开始使用我们的产品。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。''',
    llm_config=llm_config,
)

In [5]:
onboarding_topic_preference_agent = RolePlayingAgent(
    name="Onboarding Topic preference Agent",
    system_message='''您是一位乐于助人的客户引导代理，
    您在这里是为了帮助新客户开始使用我们的产品。
    
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。''',
    llm_config=llm_config,
)

In [6]:
customer_engagement_agent = RolePlayingAgent(
    name="Customer Engagement Agent",
    system_message='''您是一位乐于助人的客户服务代理
    在这里根据用户的需求为客户提供乐趣
    个人信息和主题偏好。
    这可能包括有趣的事实、笑话或有趣的故事。
    确保让它引人入胜且有趣！''',
    llm_config=llm_config,
)

我们还需要两个Agent将聊天内容整理成JSON格式结构数据

In [7]:
onboarding_personal_information_summary_agent = SummaryAgent(
    system_message="Return the customer information "
                   "into as JSON object only: "
                   "{'name': '', 'location': ''}",
    llm_config=llm_config)

In [8]:
onboarding_topic_preference_summary_agent = SummaryAgent(
    system_message="Return the customer information "
                   "into as JSON object only: "
                   "{'interests': ''}",
    llm_config=llm_config)

最后设定工作流

In [9]:
def agent_workflow(customer):
    session1 = Session(participants=[onboarding_personal_information_agent, customer],
                       termination_msg="terminate")
    session1.append(
        (onboarding_personal_information_agent, "您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？"))
    session1.chat(max_turns=5)

    name_location = onboarding_personal_information_summary_agent.summary(session1)

    print("客户的姓名和目的地:")
    print(name_location)

    session2 = Session(participants=[onboarding_topic_preference_agent, customer],
                       termination_msg="terminate")
    session2.append((onboarding_topic_preference_agent, "棒!你能告诉我你对什么主题感兴趣。"))
    session2.chat(max_turns=5)

    interests = onboarding_topic_preference_summary_agent.summary(session2)

    print("客户的感兴趣的主题:")
    print(interests)

    session3 = Session(participants=[customer_engagement_agent, customer],
                       termination_msg="terminate")
    session3.append((customer, f"客户个人信息: \n {name_location} \n\n 客户感兴趣的主题: \n {interests} \n\n"))
    session3.chat(max_turns=5)

在让正式客户使用之前，可以让大语言模型扮演客户测试一下

In [10]:
customer = RolePlayingAgent(
    name="Customer",
    system_message='''你是一名顾客。
    你的姓名叫小王，目的地是成都，你喜欢的动漫和电影。
    ''',
    llm_config=llm_config
)

In [11]:
agent_workflow(customer)

Onboarding Personal Information Agent (round 0)

您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？

Customer (round 0)

您好！我叫小王。目的地是成都。我特别喜欢动漫和电影，希望能体验到与这些文化相关的活动和景点。在成都，我想寻找一些有特色的动漫展览或是电影院体验。是否能给我一些建议？

Onboarding Personal Information Agent (round 1)

非常感谢您提供的信息，小王先生。基于您的兴趣，我为您提供以下建议，希望对您在成都的旅行有所帮助：

1. **动漫展览**：您可以参观成都东郊记忆，这里不仅有丰富的历史文化遗产，也是成都文化的聚集地。其中的“豆瓣电影院”为参观者提供了一批特色视听享乐场所。您可以关注他们的社交平台，了解更多将要举办的动漫展览活动。

2. **电影院体验**：成都的电影院资源丰富，推荐前往红星双人影城、万象影城-成都万象城店等，这些高端影城提供较为舒适的观影体验。同时，关注一票通电影小程序，这样可以快速查询和购买电影票，节省时间。

3. **动漫艺术博物馆**：虽然成都没有专门的动漫艺术博物馆，但您可以在新开业的博物馆或文化场所，参加一些相关主题的展览。比如成都市美术馆，虽然展出内容多样化，但艺术展中经常会有与流行文化、动漫艺术相关的作品。

4. **主题餐厅和活动**：在成都旅行期间，还可以尝试一些与动漫主题相结合的特色餐厅和咖啡馆，这些地点通常也会举办一些主题相关的活动和展览，增添旅行乐趣。

我所提供的这些建议希望能帮助您在成都的旅行期间，享受更多与动漫和电影相关的内容。如果需要更具体的信息或者有其他需求，请随时联系我。祝您旅途愉快！

---

完成任务，返回：“TERMINATE”。

客户的姓名和目的地:
TERMINATE
Onboarding Topic preference Agent (round 0)

棒!你能告诉我你对什么主题感兴趣。

Customer (round 0)

作为“小王”，虽然我实际上是一个AI助手，但我可以根据普遍的知识来回答你的问题。对于“主题”感兴趣的情绪文化、音乐、电影、动漫、科技、健康、旅行等领域，很多人都有独特的兴趣。如果你具体问到某个主题比如“动漫”或“

看上去不错，那就我们自己试一下吧

In [12]:
customer = UserInput()

In [13]:
agent_workflow(customer)

Onboarding Personal Information Agent (round 0)

您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？

user input: 我叫小王
user (round 0)

我叫小王

Onboarding Personal Information Agent (round 1)

你好，小王。接下来，请告诉我你的目的地是哪里？

user input: 我要去成都
user (round 1)

我要去成都

Onboarding Personal Information Agent (round 2)

好的，小王，你将前往成都。我们已经收集到了所有必要的信息。祝你旅途愉快！返回“TERMINATE”。

客户的姓名和目的地:
{"name": "小王", "location": "成都"}
Onboarding Topic preference Agent (round 0)

棒!你能告诉我你对什么主题感兴趣。

user input: 我对电影和动漫感兴趣，最近沉迷银河破裂者这款游戏
user (round 0)

我对电影和动漫感兴趣，最近沉迷银河破裂者这款游戏

Onboarding Topic preference Agent (round 1)

太棒了，关于电影、动漫和游戏你都有浓厚的兴趣。你最近有发现特别喜欢的电影、动漫作品或是银河破裂者游戏的特定玩法吗？我可以为你推荐更多相关的内容或者帮助解答你在游戏过程中遇到的问题。

user input: 给我讲讲成都有什么好玩的地方
user (round 1)

给我讲讲成都有什么好玩的地方

Onboarding Topic preference Agent (round 2)

成都是一个充满魅力的历史文化名城，下面是我为您推荐的一些必去的著名景点：

1. **武侯祠与锦里**：武侯祠纪念三国时期的政治家诸葛亮，是了解三国历史的好地方。锦里则是古色古香的商业街，可以逛逛、品尝当地小吃。

2. **成都大熊猫繁育研究基地**：这里是观看熊猫的最佳地点，不仅可以看到大熊猫，还可以学习到更多关于大熊猫保护的知识。

3. **杜甫草堂**：这个地方是用来纪念唐代著名诗人杜甫的，你可以在这里感受到唐文化的韵味和成都的诗意。

4. **

## 总结
简单把多个agent串起来就能实现很多有意思的功能